In [1]:
import requests
import json
import pandas as pd

# Generate authentication info

In [5]:
base_url = "http://localhost:8000/api/v1"  # Replace this url with the actual url of your server
token_generation_url = f"{base_url}/login/access-token"
auth_info = {
    "username": "your_username",
    "password": "your_password"
}
r = requests.post(token_generation_url, data=auth_info)
access_token = r.json()['access_token']
headers = {
    'accept': 'application/json',
    'Authorization': 'Bearer ' + access_token
}

# Query data from DB

### Retrive data from DB using table_name

In [8]:
table_name = 'exampletable4'
retrieve_data_api = f"{base_url}/queries/{table_name}"
payload = {'skip': 0, 'limit': 5}
r = requests.get(retrieve_data_api, params=payload, headers=headers)
r.json()

[{'id': 1,
  'sample_col1': 'a',
  'sample_col2': 1,
  'sample_col3': 1,
  'sample_col4': -1,
  'sample_col5': -0.1},
 {'id': 2,
  'sample_col1': 'b',
  'sample_col2': 2,
  'sample_col3': 4,
  'sample_col4': -4,
  'sample_col5': -0.4},
 {'id': 3,
  'sample_col1': 'c',
  'sample_col2': 3,
  'sample_col3': 9,
  'sample_col4': -9,
  'sample_col5': -0.9},
 {'id': 4,
  'sample_col1': 'd',
  'sample_col2': 4,
  'sample_col3': 16,
  'sample_col4': -16,
  'sample_col5': -1.6},
 {'id': 5,
  'sample_col1': 'e',
  'sample_col2': 5,
  'sample_col3': 25,
  'sample_col4': -25,
  'sample_col5': -2.5}]

### Retrieve column names for a table

In [9]:
retrieve_column_names_api = f"{base_url}/queries/{table_name}/column-names"
r = requests.get(retrieve_column_names_api, headers=headers)
r.json()

{'column_names': ['id',
  'sample_col1',
  'sample_col2',
  'sample_col3',
  'sample_col4',
  'sample_col5']}

### Only retrieve selected columns

In [10]:
payload = {'attr_names': ['sample_col1', 'sample_col2'], 'skip': 0, 'limit': 5}
r = requests.get(retrieve_data_api, params=payload, headers=headers)
r.json()

[{'sample_col1': 'a', 'sample_col2': 1},
 {'sample_col1': 'b', 'sample_col2': 2},
 {'sample_col1': 'c', 'sample_col2': 3},
 {'sample_col1': 'd', 'sample_col2': 4},
 {'sample_col1': 'e', 'sample_col2': 5}]

### Query data from APIs and convert into a DataFrame

In [12]:
total_num_of_records = r.json()['total_num_of_records']
skip = 0
limit = 1000
df = pd.DataFrame()
# Query several time to aviod too large data transfer issue.
while skip < total_num_of_records:
    payload = {'skip': skip, 'limit': limit}
    r = requests.get(retrieve_data_api, params=payload, headers=headers)
    if r.status_code == 200:
        df = df.append(pd.DataFrame(r.json()))
    else:
        print("Data query failed!")
    skip = skip + limit

In [13]:
df

,id,sample_col1,sample_col2,sample_col3,sample_col4,sample_col5
0,1,a,1,1,-1,-0.1
1,2,b,2,4,-4,-0.4
2,3,c,3,9,-9,-0.9
3,4,d,4,16,-16,-1.6
4,5,e,5,25,-25,-2.5
5,6,f,6,36,-36,-3.6
6,7,g,7,49,-49,-4.9
